In [15]:
import pandas as pd

df = pd.read_csv('playbook.csv', parse_dates=True, names=['contract', 'direction', 'vol'])
df

,contract,direction,vol
2021-08-30,510050C2109M03000,L,20
2021-08-30,510050C2109M03000,S,20
2021-08-30,510050C2110M02800,L,50
2021-08-30,510050C2110M03000,S,50
2021-08-30,M0C5,L,20
2021-08-30,M0C6,S,20
2021-08-30,M1C4,L,50
2021-08-30,M1C6,S,50


In [32]:
time = pd.Timestamp('20010101')
time + pd.Timedelta('1d')
for row in df[df.index == pd.Timestamp('20210830')].itertuples(index=False):
    print(row.contract, row.direction)

 510050C2109M03000  L
 510050C2109M03000  S
 510050C2110M02800  L
 510050C2110M03000  S
 M0C5  L
 M0C6  S
 M1C4  L
 M1C6  S


In [207]:
class Condition(dict):
    def __init__(self, type, **kargs) -> None:
        self.type = type
        super().__init__(kargs)

c = Condition('a', **{'x':1, 'y':2})
c

{'x': 1, 'y': 2}

In [239]:
import numpy as np
dfa = pd.DataFrame(np.arange(100, 110), pd.date_range('20000101', periods=10, freq='D'), columns=['A'])
dfb = pd.DataFrame(np.arange(200, 210), pd.date_range('20000101', periods=10, freq='D'), columns=['A'])
concated = pd.concat([dfa, dfb], axis=1, keys=['a', 'b'])
print(concated)
dfc = pd.DataFrame({'a': range(300,310), 'b': pd.date_range('20000101', periods=10)})
dfc = dfc.set_index('b')
joined = concated.join(dfc)
joined.columns


              a    b
              A    A
2000-01-01  100  200
2000-01-02  101  201
2000-01-03  102  202
2000-01-04  103  203
2000-01-05  104  204
2000-01-06  105  205
2000-01-07  106  206
2000-01-08  107  207
2000-01-09  108  208
2000-01-10  109  209


C:\Users\xniu\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning:

merging between different levels can give an unintended result (2 levels on the left,1 on the right)



Index([('a', 'A'), ('b', 'A'), 'a'], dtype='object')

In [47]:
def atm(spot_price:float):
    x = round(spot_price, 1)
    if x < 3.0:
        mid = x + (0.05 if x < spot_price else -0.05)
        return mid if abs(spot_price - x) > abs(spot_price - mid) else x
    else: 
        return x

atm(2.283)


2.3

In [194]:
from collections import namedtuple
Trade = namedtuple('Trade', 'name symbol direction vol date trigger', defaults=['', '', 'L', 0, None, None])
d = Trade._field_defaults
d['a'] = 0
Trade(**d)

TypeError: __new__() got an unexpected keyword argument 'a'

In [214]:
import yaml
with open('playbook.yml') as f:
    res = yaml.safe_load(f)
    print(res['accounts'][1]['trades'][1]['closePolicy'])
    print(res['accounts'][1]['trades'][1])

[{'type': 'pl', 'expression': '$>0.8'}, {'type': 'linked', 'position': 'primary', 'on': 'close'}]
{'date': datetime.date(2022, 3, 25), 'symbol': '510050C2204M02900', 'direction': 'S', 'volume': 20, 'closePolicy': [{'type': 'pl', 'expression': '$>0.8'}, {'type': 'linked', 'position': 'primary', 'on': 'close'}]}


In [130]:
import re
m = re.match(r'{([CP]\dM\d)}', '{C6S1}')
print(m)

m = re.match(r'510050[CP]\d{4}M\d{5}', '510050C2203M03600')
print(m)

None
<re.Match object; span=(0, 17), match='510050C2203M03600'>


In [100]:
import pandas as pd
date = pd.Timestamp('20220330')
x = pd.offsets.QuarterEnd(2, startingMonth=3)
print(date + x)
print(x.rollforward(date))
print(x.rollforward(date) + x)

print(pd.Timestamp('20220330') + pd.offsets.MonthEnd(2))
print(pd.Timestamp('20220331') + pd.offsets.MonthEnd(2))

print(f"{date.strftime('%y%m')}")

2022-06-30 00:00:00
2022-03-31 00:00:00
2022-09-30 00:00:00
2022-04-30 00:00:00
2022-05-31 00:00:00
2203


In [132]:
import akshare as ak
ak.option_sse_daily_sina(symbol='10003749')

,日期,开盘,最高,最低,收盘,成交量
0,2021-11-29,0.1971,0.2008,0.1766,0.1972,30513774
1,2021-11-30,0.2008,0.2084,0.1682,0.1801,39758101
2,2021-12-01,0.1801,0.1952,0.1729,0.1898,67418913
3,2021-12-02,0.1851,0.2081,0.1776,0.2003,140634378
4,2021-12-03,0.2045,0.2237,0.1979,0.2207,58243454
5,2021-12-06,0.2292,0.2548,0.2275,0.2332,59529638
6,2021-12-07,0.2473,0.2637,0.2388,0.2536,50294197
7,2021-12-08,0.2536,0.3072,0.2474,0.3060,40615443
8,2021-12-09,0.3188,0.3980,0.3082,0.3570,39641041
9,2021-12-10,0.3470,0.3588,0.3409,0.3497,9573629


In [131]:
import replay
import hist_analysis
import importlib
importlib.reload(replay)
hist_analysis.History.loaddata()
replay.Account.resolveContract('P9M3', '20211220')

atm:3.2
step 0.1 => 3.3000000000000003
step 0.1 => 3.4000000000000004
step 0.1 => 3.5000000000000004
step 0.1 => 3.6000000000000005
dynmaic name 'P9M3 on 20211220 resolved to 510050P2203M03600


'510050P2203M03600'

In [183]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

replay_df = pd.read_csv("res.csv", parse_dates=True)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
    specs=[
        [{'secondary_y':True}], 
        [{}]])

spot_tr = go.Scatter(x=replay_df.index, y=replay_df['spot'], name='spot')
pl_amt_tr = go.Scatter(x=replay_df.index, y=replay_df['pl_amt'], name='p&l-amount')
fig.add_traces([spot_tr, pl_amt_tr], rows=1, cols=1, secondary_ys=[True, False])

pl_pct_tr = go.Scatter(x=replay_df.index, y=replay_df['pl_pct']*100, name='p&l-percent')
delta_tr = go.Scatter(x=replay_df.index, y=replay_df['delta'], name='delta')
long_vol_tr = go.Bar(x=replay_df.index, y=replay_df['long_vol'], name='long vol')
short_vol_tr = go.Bar(x=replay_df.index, y=replay_df['short_vol'], name='short vol')
fig.add_traces([pl_pct_tr, delta_tr, long_vol_tr, short_vol_tr], rows=2, cols=1)

fig['data'][2]['yaxis'] = 'y4'
fig['data'][3]['yaxis'] = 'y5'
fig['layout']['yaxis4']=dict(
        title="pl",
        overlaying="y3",
        anchor="x2",
        side="right"
        #position=0.15
    )
fig['layout']['yaxis5']=dict(
        title="delta",
        anchor="free",
        overlaying='y3',
        range=[-1, 1],
        side="right",
        position=1
    )
fig.update_layout(barmode='stack', height=600, title='replay')
fig.show()

In [184]:

print(fig['layout'])

for i in range(len(fig['data'])):
    print(i, fig['data'][i].name, fig['data'][i]['xaxis'], fig['data'][i]['yaxis'])

Layout({
    'barmode': 'stack',
    'height': 600,
    'template': '...',
    'title': {'text': 'replay'},
    'xaxis': {'anchor': 'y', 'domain': [0.0, 0.94], 'matches': 'x2', 'showticklabels': False},
    'xaxis2': {'anchor': 'y3', 'domain': [0.0, 0.94]},
    'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0]},
    'yaxis2': {'anchor': 'x', 'overlaying': 'y', 'side': 'right'},
    'yaxis3': {'anchor': 'x2', 'domain': [0.0, 0.425]},
    'yaxis4': {'anchor': 'x2', 'overlaying': 'y3', 'side': 'right', 'title': {'text': 'pl'}},
    'yaxis5': {'anchor': 'free',
               'overlaying': 'y3',
               'position': 1,
               'range': [-1, 1],
               'side': 'right',
               'title': {'text': 'delta'}}
})
0 spot x y2
1 p&l-amount x y
2 p&l-percent x2 y4
3 delta x2 y5
4 long vol x2 y3
5 short vol x2 y3
